https://www.kaggle.com/artgor/dcgan-baseline/notebook
https://www.kaggle.com/jesucristo/gan-introduction
https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html?source=post_page---------------------------
https://github.com/znxlwm/pytorch-MNIST-CelebA-GAN-DCGAN

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
import pickle as pkl
%matplotlib inline
import torchvision.utils as vutils


In [ ]:

batch_size = 128
z_size = 100 # Size of z latent vector (i.e. size of generator input)
img_size = 64


In [ ]:
def get_dataloader(batch_size, image_size, data_dir=r'../input/img_align_celeba'):
  
    transform = transforms.Compose([transforms.Resize((image_size, image_size)), 
                                transforms.CenterCrop(image_size),
                                transforms.ToTensor(), 
                                transforms.Normalize([0.5, 0.5, 0.5],
                                                    [0.5, 0.5, 0.5])])
                              
                                
    dataset=datasets.ImageFolder(data_dir,transform)
    dataloader=torch.utils.data.DataLoader(dataset=dataset,batch_size=batch_size)
    
    return dataloader

# Call your function and get a dataloader
celeba_train_loader = get_dataloader(batch_size, img_size)

In [ ]:
## Method to display Image for Tensor
def imshow(image, ax=None, title=None, normalize=True):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    image = image.numpy().transpose((1, 2, 0))

    if normalize:
        mean = np.array([0.5, 0.5, 0.5])
        std = np.array([0.5, 0.5, 0.5])
        image = std * image + mean
        image = np.clip(image, 0, 1)

    ax.imshow(image)
#     ax.spines['top'].set_visible(False)
#     ax.spines['right'].set_visible(False)
#     ax.spines['left'].set_visible(False)
#     ax.spines['bottom'].set_visible(False)
#     ax.tick_params(axis='both', length=0)
    ax.set_xticklabels('')
    ax.set_yticklabels('')
    return ax

#dataiter = iter(celeba_train_loader)
images, labels = next(iter(celeba_train_loader))

fig, axes = plt.subplots(figsize=(16,8), ncols=8)
for ii in range(8):
    ax = axes[ii]
    imshow(images[ii], ax=ax, normalize=True)       

### Initializing Weights

In [ ]:
#Initializing weights
def init_weight(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(0.1, 0.02)
        m.bias.data.fill_(0)

### Discriminator

In [ ]:
# Discriminator
class Discriminator(nn.Module):
    
    def __init__(self, inhw, c1_channels=64, c2_channels=128, c3_channels=256,
                 c4_channels=512, i_channels_in_2=True):
        '''
        The constructor method for the Discriminator class
        
        Arguments:
        - inhw : The number of 
        - c1_channels : the number of output channels from the
                        first Convolutional Layer [Default - 128]
                        
        - c2_channels : the number of output channels from the
                        second Convolutional Layer [Default - 256]
                        
        - c3_channels : the number of output channels from the
                        third Convolutional Layer [Default - 512]
        
        - i_channels_in_2 : Increase the number of channels by 2
                        in each layer.
        '''
        
        super().__init__()
        
        # Define the class variables
        self.c1_channels = c1_channels
        
        if i_channels_in_2:
            self.c2_channels = self.c1_channels * 2
            self.c3_channels = self.c2_channels * 2
            self.c4_channels = self.c3_channels * 2
        else:
            self.c2_channels = c2_channels
            self.c3_channels = c3_channels
            self.c4_channels = c4_channels
        
        self.conv1 = nn.Conv2d(in_channels=3,
                               out_channels=self.c1_channels,
                               kernel_size=4,
                               stride=2,
                               padding=1,
                               bias=False)
        
        self.conv2 = nn.Conv2d(in_channels=self.c1_channels,
                               out_channels=self.c2_channels,
                               kernel_size=4,
                               stride=2,
                               padding=1,
                               bias=False)
        
        self.bnorm2 = nn.BatchNorm2d(num_features=self.c2_channels)
        
        self.conv3 = nn.Conv2d(in_channels=self.c2_channels,
                               out_channels=self.c3_channels,
                               kernel_size=4,
                               stride=2,
                               padding=1,
                               bias=False)
        
        self.bnorm3 = nn.BatchNorm2d(num_features=self.c3_channels)
        
        self.conv4 = nn.Conv2d(in_channels=self.c3_channels,
                               out_channels=self.c4_channels,
                               kernel_size=4,
                               stride=2,
                               padding=1,
                               bias=False)
        
        self.bnorm4 = nn.BatchNorm2d(num_features=self.c4_channels)
        
        self.conv5 = nn.Conv2d(in_channels=self.c4_channels,
                               out_channels=1,
                               kernel_size=4,
                               padding=0,
                               stride=1,
                               bias=False)
        
        self.lrelu = nn.LeakyReLU(negative_slope=0.2)
        
        self.sigmoid = nn.Sigmoid()
        
        
    def forward(self, img):
        '''
        The method for the forward pass in the network
        
        Arguments;
        - img : a torch.tensor that is of the shape N x C x H x W
                where, N - the batch_size
                       C - the number of channels
                       H - the height
                       W - the width
       
       Returns:
       - out : the output of the Discriminator 
               whether the passed image is real /fake
        '''      
        batch_size = img.shape[0]
        
        x = self.lrelu(self.conv1(img))
        x = self.lrelu(self.bnorm2(self.conv2(x)))
        x = self.lrelu(self.bnorm3(self.conv3(x)))
        x = self.lrelu(self.bnorm4(self.conv4(x)))
        x = self.conv5(x)
        
        x = self.sigmoid(x)
        
        return x.view(-1, 1).squeeze()
      
    def out_shape(self, inp_dim, kernel_size=4, padding=1, stride=2):
        return ((inp_dim - kernel_size + (2 * padding)) // stride) + 1

In [ ]:
# apply weights to discriminator
dis = Discriminator(64).cuda()
dis.apply(init_weight)
print (dis)

### Generator

In [ ]:
# Generator
class Generator(nn.Module):
    def __init__(self, ct1_channels=512, ct2_channels=256,
                 ct3_channels=128, ct4_channels=64, d_channels_in_2=False):
        
        '''
        The contructor class for the Generator
        
        Arguments:
        - zin_channels: ###
        
        - ct1_channels: The number of output channels for the
                        first ConvTranspose Layer. [Default - 1024]
        
        - ct2_channels: The number of putput channels for the
                        second ConvTranspose Layer. [Default - 512]
                        
        - ct3_channels: The number of putput channels for the
                        third ConvTranspose Layer. [Default - 256]
                        
        - ct4_channels: The number of putput channels for the
                        fourth ConvTranspose Layer. [Default - 128]
                        
        - d_channnels_in_2 : Decrease the number of channels 
                        by 2 times in each layer.
                        
        '''
        super().__init__()
        
        # Define the class variables
        self.ct1_channels = ct1_channels
        self.pheight = 4
        self.pwidth = 4
        
        if d_channels_in_2:
            self.ct2_channels = self.ct1_channels // 2
            self.ct3_channels = self.ct2_channels // 2
            self.ct4_channels = self.ct3_channels // 2
        else:
            self.ct2_channels = ct2_channels
            self.ct3_channels = ct3_channels
            self.ct4_channels = ct4_channels
        
        self.convt_0 = nn.ConvTranspose2d(in_channels=z_size,
                                          out_channels=self.ct1_channels,
                                          kernel_size=4,
                                          padding=0,
                                          stride=1,
                                          bias=False)
        
        self.bnorm0 = nn.BatchNorm2d(self.ct1_channels)
        
        self.convt_1 = nn.ConvTranspose2d(in_channels=self.ct1_channels,
                                          out_channels=self.ct2_channels,
                                          kernel_size=4,
                                          stride=2,
                                          padding=1,
                                          bias=False)
        
        self.bnorm1 = nn.BatchNorm2d(num_features=self.ct2_channels)
        
        self.convt_2 = nn.ConvTranspose2d(in_channels=self.ct2_channels,
                                          out_channels=self.ct3_channels,
                                          kernel_size=4,
                                          stride=2,
                                          padding=1,
                                          bias=False)
        
        self.bnorm2 = nn.BatchNorm2d(num_features=self.ct3_channels)
        
        self.convt_3 = nn.ConvTranspose2d(in_channels=self.ct3_channels,
                                          out_channels=self.ct4_channels,
                                          kernel_size=4,
                                          stride=2,
                                          padding=1,
                                          bias=False)
        
        self.bnorm3 = nn.BatchNorm2d(num_features=self.ct4_channels)
        
        self.convt_4 = nn.ConvTranspose2d(in_channels=self.ct4_channels,
                                          out_channels=3,
                                          kernel_size=4,
                                          stride=2,
                                          padding=1,
                                          bias=False)
        
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        
    def forward(self, z):
        '''
        The method for the forward pass for the Generator
        
        Arguments:
        - z : the input random uniform vector sampled from uniform distribution
        
        Returns:
        - out : The output of the forward pass through the network
        '''
        
        # Project the input z and reshape
        x = self.relu(self.bnorm0(self.convt_0(z)))
        #print (x.shape)
        x = self.relu(self.bnorm1(self.convt_1(x)))
        x = self.relu(self.bnorm2(self.convt_2(x)))
        x = self.relu(self.bnorm3(self.convt_3(x)))
        out = self.tanh(self.convt_4(x))
        
        return out

In [ ]:
# apply weights to generator
gen = Generator().cuda()
gen.apply(init_weight)
print (gen)

Creating criterion and optimizeers

In [ ]:
# Optimizers
criterion = nn.BCELoss()
d_lr = 0.0002
g_lr = 0.0002

d_opt = optim.Adam(dis.parameters(), lr=d_lr, betas=[0.5, 0.999])
g_opt = optim.Adam(gen.parameters(), lr=g_lr, betas=[0.5, 0.999])

### Training our GAN

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Train loop
p_every = 300
t_every = 1
e_every = 1
s_every = 1
epochs = 50

# Definig labels for real (1) and fake (0) images
real_label = 1
fake_label = 0

train_losses = []
eval_losses = []
samples=[]
sample_size=16
fixed_z = np.random.uniform(-1, 1, size=(sample_size, z_size))
fixed_z = torch.from_numpy(fixed_z).float()
fixed_z = fixed_z.cuda()

for e in range(epochs):
    td_loss = 0
    tg_loss = 0
    
    for batch_i, (real_images, _) in enumerate(celeba_train_loader):      
        real_images = real_images.cuda()
        
        batch_size = real_images.size(0)
        
        #### Train the Discriminator ####
        
        ## Train with all-real batch
        # Feed forward real images in discriminator
        d_opt.zero_grad()        
        d_real = dis(real_images)        
        label = torch.full((batch_size,), real_label, device='cuda')
        #calculate loss on all-real batch
        real_loss = criterion(d_real,label)
        real_loss.backward()
        
        ## Train with all-fake batch
        # Random normal distribution for each image
        z = torch.randn(batch_size, z_size, 1, 1, device='cuda')
        fake_images = gen(z) # Generate fake images through the generat       
        label.fill_(fake_label) #label has now been filled with the values fake label
        # Feed forward in discriminator
        d_fake = dis(fake_images.detach())
        #calculate loss on all-fake batch
        fake_loss = criterion(d_fake,label)
        fake_loss.backward()
        #Add loss for both fake and real images
        d_loss = real_loss + fake_loss
        d_opt.step()
        
        
        #### Train the Generator ####
        # Feed forward for generator
        g_opt.zero_grad()        
        label.fill_(real_label)        
        d_fake2 = dis(fake_images) 
        # loss function of generator
        g_loss = criterion(d_fake2, label)
        g_loss.backward()
        # backward propagation for generator
        g_opt.step()
        
        # Plotting fake images generated after each epoch by generator
        if batch_i % p_every == 0:
          noise = torch.randn(1, 100, 1, 1, device='cuda')
          out = gen(noise)
          out = out.detach().cpu().squeeze(0).transpose(0, 1).transpose(1, 2).numpy()
          out = out * (0.5, 0.5, 0.5)
          out += (0.5, 0.5, 0.5)
          plt.axis('off')
          plt.imshow(out)
          plt.show()
          print ('Epoch [{:5d} / {:5d}] | d_loss: {:6.4f} | g_loss: {:6.4f}'. \
                    format(e+1, epochs, d_loss, g_loss))
        train_losses.append([d_loss, g_loss])
    
       
print ('Training Completed successfully!')
    
# finally return losses
#return train_losses

# Save training generator samples
#...

### Plot Discriminator and Generator Losses

In [ ]:
len(train_losses[0])
# fig, ax = plt.subplots()
# losses = np.array(train_losses)
# plt.plot(losses.T[0], label='Discriminator')
# plt.plot(losses.T[1], label='Generator')
# plt.title('Training Losses')
# plt.legend()
# plt.show()

### Save and load the pretrained model 

In [ ]:
torch.save(gen.state_dict(),'celeba_generator.pt')

gen.load_state_dict(torch.load('celeba_generator.pt'))


### Use the Generator to create some random images

In [ ]:
noise = torch.randn(1, 100, 1, 1, device='cuda')
out = gen(noise)
out = out.detach().cpu().squeeze(0).transpose(0, 1).transpose(1, 2).numpy()
out = out * (0.5, 0.5, 0.5)
out += (0.5, 0.5, 0.5)
plt.axis('off')
plt.imshow(out)
#imshow(out, ax=ax, normalize=True)    
plt.show()

### View the Generated Samples

In [ ]:
def view_samples(images):
    if type(images) == torch.tensor:
        images = images.cpu().numpy()  
        
    fig, axes = plt.subplots(figsize=(30, 30), nrows=8, ncols=8, sharey=True, sharex=True)
    
    for ax, img in zip(axes.flatten(), images):
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
        ax.imshow(img.astype(np.uint8))

In [ ]:
ff = f.detach().cpu().transpose(1, 2).transpose(2, 3).numpy()
ff2 = inputs.detach().cpu().transpose(1, 2).transpose(2, 3).numpy()

view_samples(ff)


### Save the models

In [ ]:
d_ckpt = {
    'model_state_dict' : D.state_dict(),
    'opt_state_dict' : d_opt.state_dict()
}

g_ckpt = {
    'model_state_dict' : G.state_dict(),
    'opt_state_dict' : g_opt.state_dict()
}

In [ ]:
torch.save(d_ckpt, 'd_ckpt.pth')
torch.save(g_ckpt, 'g_ckpt.pth')